In [1]:
!pip install fastapi uvicorn pyngrok nest-asyncio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 2.1 MB/s eta 0:00:00


In [2]:
import base64
import requests
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from PIL import Image
from io import BytesIO
from pyngrok import ngrok
import nest_asyncio
import uvicorn
import threading

In [ ]:
#import getpass
#api_key = getpass.getpass('Enter your API Key')
#sk-MIVTelI95ZSWLOQDUTSJJQWGGGdq6wK9V9hWNeQh0czeba1Y

In [ ]:
#"An Image of a beautiful chubby lady, black long hair, fair color, gazing eyes, adorable nose, sharp and thick eyebrows, dimple on cheeks, beautiful and obvious cleavage, curvy, beautiful feet, black short dress"

In [3]:
# Initialize FastAPI app
app = FastAPI()

# Initialize ngrok and nest_asyncio
nest_asyncio.apply()

# Define the input schema using Pydantic
class PromptRequest(BaseModel):
    prompt: str

# Define the API Key
api_key = "sk-MIVTelI95ZSWLOQDUTSJJQWGGGdq6wK9V9hWNeQh0czeba1Y"

# Function to generate image from prompt
def generate_image_from_prompt(prompt):
    if api_key is None:
        raise HTTPException(status_code=401, detail="Missing API key.")

    response = requests.post(
        f"https://api.stability.ai/v1/generation/stable-diffusion-v1-6/text-to-image",
        headers={
            "Content-Type": "application/json",
            "Accept": "application/json",
            "Authorization": f"Bearer {api_key}"
        },
        json={
            "text_prompts": [{"text": prompt}],
            "cfg_scale": 7,
            "height": 1024,
            "width": 1024,
            "samples": 1,
            "steps": 30,
        },
    )

    if response.status_code != 200:
        raise HTTPException(status_code=response.status_code, detail=response.text)

    data = response.json()
    image_data = base64.b64decode(data["artifacts"][0]["base64"])
    return image_data

In [4]:
# FastAPI endpoint that accepts a prompt and returns an image
@app.post("/generate_image")
async def generate_image(prompt_request: PromptRequest):
    try:
        image_data = generate_image_from_prompt(prompt_request.prompt)

        # Convert image bytes to a PIL image and then back to base64
        image = Image.open(BytesIO(image_data))
        buffered = BytesIO()
        image.save(buffered, format="PNG")
        img_str = base64.b64encode(buffered.getvalue()).decode('utf-8')

        # Return the image in base64 format
        return {"image": img_str}

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))



In [5]:
# Setup pyngrok for public URL
NGROK_AUTHTOKEN = "2mi6mm6o9xj60W1YZSvS95JqfPC_4AdAxW5jQKTUHuatgoKh1"
!ngrok authtoken {NGROK_AUTHTOKEN}

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [6]:
def run_app():
    uvicorn.run(app, host="0.0.0.0", port=8000)

thread = threading.Thread(target=run_app)
thread.start()


INFO:     Started server process [571]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


In [7]:
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")

Public URL: NgrokTunnel: "https://604f-34-75-155-252.ngrok-free.app" -> "http://localhost:8000"
